In [1]:
import datetime
from pycti import OpenCTIApiClient
from dateutil.parser import parse
import pandas as pd
import os
import warnings

In [107]:
api_url = "http://192.168.1.251:8080"
api_token = "b939cbc2-09b0-4282-bdd7-126162946ba3"

# OpenCTI initialization
opencti_api_client = OpenCTIApiClient(api_url, api_token)


INFO:root:Listing Threat-Actors with filters null.


In [3]:
api_url = "https://demo.opencti.io"
api_token = "2b4f29e3-5ea8-4890-8cf5-a76f61f1e2b2"

# OpenCTI initialization
opencti_api_client = OpenCTIApiClient(api_url, api_token)


INFO:root:Listing Threat-Actors with filters null.


In [126]:
class entity_insertion:
    
   
        

    def __init__(self,report_path,relation_path,stix_domain= dict(),ext_ref=dict()):
    
        self.report_path = report_path
        self.stix_domain = stix_domain
        self.ext_ref = ext_ref
        self.relation_path =relation_path
    
    def add_mark_def(self,row,stix_id):
        mark      =row['marking_definitions']
        all_marks =opencti_api_client.marking_definition.list()
        for i in all_marks:
            if mark in i['definition'].lower():
                mark_code= i['id']
                opencti_api_client.stix_entity.add_marking_definition(id=stix_id,marking_definition_id=mark_code)  
                
    def add_external_references_to_stix_entity(self,url,entity_id,ext_ref):
        
        key=ext_ref.keys()
        if url in key :
                opencti_api_client.stix_entity.add_external_reference(id=entity_id, external_reference_id= self.stix_domain[url]
                                                                     )
    def creating_and_adding_ext_ref(self,by,report):
        
        ext_ref_details = pd.read_excel(self.report_path,sheet_name = 'external_ref')
        ext_ref_details = ext_ref_details.where(ext_ref_details.notnull(), None)
        
        for index,row in ext_ref_details.iterrows():
                
                external_reference = opencti_api_client.external_reference.create(
                            source_name= row['source_name'],
                            url= row['url'].strip(),
                            external_id =row['external_id'],
                            description = row['description']
                )
                    
                opencti_api_client.stix_entity.add_external_reference(id=report,external_reference_id = external_reference['id'] )  
                self.ext_ref.update ({row['url'].strip() : external_reference['id']})               
     
    def create_threat_actor(self,by,report):
            
            threat_actor_details = pd.read_excel(self.report_path,sheet_name = 'threat_actor')
            threat_actor_details = threat_actor_details.where(threat_actor_details.notnull(), None)
            
            for index,row in threat_actor_details.iterrows():
            
                print(row['name'])
                if row['alias'] is not None:
                    aliases =row['alias'].split(',')
                else:
                    aliases = None
                    

                
                threat_actor = opencti_api_client.threat_actor.create(
                                                name = row['name'].strip(),
                                                description = row['description'],
                                                alias = aliases ,
                                                goal = row['goal'] ,
                                                sophistication = row['sophistication'] ,
                                                resource_level = row['resource_level'] ,
                                                primary_motivation = row['primary_motivation'] ,
                                                secondary_motivation = row['secondary_motivation'] ,
                                                personal_motivation = row['personal_motivation'] ,
                                                modified = datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00") ,
                                                createdByRef = by,
                                                update = False
                                                 )
                    
                if row['marking_definitions'] is not None:
                        self.add_mark_def(stix_id=threat_actor['id'],row=row)
                        
                opencti_api_client.report.add_stix_entity(id=report,entity_id = threat_actor['id'] )       
                self.stix_domain.update ({row['name'].strip() : threat_actor['id']})
                self.add_external_references_to_stix_entity (url=row['external_references'],entity_id =threat_actor['id'],ext_ref =self.ext_ref )
                
                        
    def create_attack_pattern(self,by,report):           #attack_pattern insertion    
            
            attack_pattern_details = pd.read_excel(self.report_path,sheet_name = 'attack_pattern')
            attack_pattern_details = attack_pattern_details.where(attack_pattern_details.notnull(), None)
            
            for index,row in attack_pattern_details.iterrows():
            
                if row['alias'] is not None:
                    aliases =row['alias'].split(',')
                else:
                    aliases = None
                    
                if row['platform'] is not None:
                    platforms =row['platform'].split(',')
                else:
                    platforms = None
                    
                print(row['required_permission'])   
                
                exist = opencti_api_client.attack_pattern.list ( filters=[{'key' : 'name' , 'values' : row['name']}])
                
                if (len(exist) == 0):
                    by = by
                else:
                    by = None
                
                
                attack_pattern = opencti_api_client.attack_pattern.create(
                                                name = row['name'].strip(),
                                                description = row['description'],
                                                alias = aliases,
                                                platform  = platforms ,
                                                required_permission = row['required_permission'] ,
                                                external_id  = row['external_id'] ,
                                                modified = datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00") ,
                                                createdByRef = by,
                                                update=True
                                                )
                                          
                if row['marking_definitions'] is not None:
                        self.add_mark_def(stix_id=attack_pattern['id'],row=row)
                        
                        
                opencti_api_client.report.add_stix_entity(id=report,entity_id = attack_pattern['id'] )  
                self.stix_domain.update ({row['name'].strip() : attack_pattern['id']})      
                self.add_external_references_to_stix_entity (url=row['external_references'],entity_id =attack_pattern['id'],ext_ref =self.ext_ref ) 
                    
    def create_malware(self,by,report):           #malware insertion    
            
            malware_details = pd.read_excel(self.report_path,sheet_name = 'malware')
            malware_details = malware_details.where(malware_details.notnull(), None)
            
            for index,row in malware_details.iterrows():
            
                if row['alias'] is not None:
                    aliases =row['alias'].split(',')
                    print(aliases)
                else:
                    aliases = None
                
                exist = opencti_api_client.malware.list ( filters=[{'key' : 'name' , 'values' : row['name']}])
            
                if (len(exist) == 0):
                    malware = opencti_api_client.malware.create(
                                                name = row['name'].strip(),
                                                description = row['description'],
                                                alias = aliases ,
                                                modified = datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00") ,
                                                createdByRef = by,
                                                )
                else:
                     malware = opencti_api_client.malware.create(
                                                name = row['name'].strip(),
                                                description = row['description'],
                                                alias = aliases ,
                                                modified = datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00") ,
                                                update = True,
                                                 )
                if row['marking_definitions'] is not None:
                        self.add_mark_def(stix_id=malware['id'],row=row)
                        
                opencti_api_client.report.add_stix_entity(id=report,entity_id = malware['id'] )
                self.stix_domain.update ({row['name'].strip() : malware['id']}) 
                self.add_external_references_to_stix_entity (url=row['external_references'],entity_id =malware['id'],ext_ref =self.ext_ref )
                
    def create_tool(self,by,report):           #malware insertion    
            
            tool_details = pd.read_excel(self.report_path,sheet_name = 'tool')
            tool_details = tool_details.where(tool_details.notnull(), None)
            
            for index,row in tool_details.iterrows():
            
                if row['alias'] is not None:
                    aliases =row['alias'].split(',')
                    print(aliases)
                else:
                    aliases = None
                
                exist = opencti_api_client.tool.list ( filters=[{'key' : 'name' , 'values' : row['name']}])
            
                if (len(exist) == 0):
                    tool = opencti_api_client.tool.create(
                                                name = row['name'].strip(),
                                                description = row['description'],
                                                alias = aliases ,
                                                modified = datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00") ,
                                                createdByRef = by,
                                                )
                else:
                     tool = opencti_api_client.tool.create(
                                                name = row['name'].strip(),
                                                description = row['description'],
                                                alias = aliases ,
                                                modified = datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00") ,
                                                update = True,
                                                 )
                if row['marking_definitions'] is not None:
                        self.add_mark_def(stix_id=tool['id'],row=row)
                        
                opencti_api_client.report.add_stix_entity(id=report,entity_id = tool['id'] )
                self.stix_domain.update ({row['name'].strip() : tool['id']})
                self.add_external_references_to_stix_entity (url=row['external_references'],entity_id =tool['id'],ext_ref =self.ext_ref )
                
    def create_vulnerabilty(self,by,report):           #vulnerability insertion    
            
            vul_details = pd.read_excel(self.report_path,sheet_name = 'vulnerability')
            vul_details = vul_details.where(vul_details.notnull(), None)
            
            for index,row in vul_details.iterrows():
            
                if row['alias'] is not None:
                    aliases =row['alias'].split(',')
                    print(aliases)
                else:
                    aliases = None
                
                exist = opencti_api_client.vulnerability.list ( filters=[{'key' : 'name' , 'values' : row['name']}])
            
                if (len(exist) == 0):
                    vulnerability = opencti_api_client.vulnerability.create(
                                                name = row['name'].strip(),
                                                description = row['description'],
                                                alias = aliases ,
                                                modified = datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00") ,
                                                createdByRef = by,
                                                )
                else:
                     vulnerability = opencti_api_client.vulnerability.create(
                                                name = row['name'].strip(),
                                                description = row['description'],
                                                alias = aliases ,
                                                modified = datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00") ,
                                                update = True,
                                                 )
                if row['marking_definitions'] is not None:
                        self.add_mark_def(stix_id=vulnerability['id'],row=row)
                        
                opencti_api_client.report.add_stix_entity(id=report,entity_id = vulnerability['id'] )
                self.stix_domain.update ({row['name'].strip() : vulnerability['id']})  
                self.add_external_references_to_stix_entity (url=row['external_references'],entity_id =vulnerability['id'],ext_ref =self.ext_ref )
                
    def create_indicator(self,by,report):           #vulnerability insertion    
            
            indicator_details = pd.read_excel(self.report_path,sheet_name = 'Indicator')
            indicator_details = indicator_details.where(indicator_details.notnull(), None)
            
            for index,row in indicator_details.iterrows():
            
                #exist = opencti_api_client.indicator.list ( filters=[{'key' : 'name' , 'values' : row['name']}])
                exist=[]
                if (len(exist) == 0):
                    indicator = opencti_api_client.indicator.create(
                                                                name=row['name'],
                                                                description= row['description'],
                                                                pattern_type= row['pattern_type'],
                                                                main_observable_type= row ['main_observable_type'],
                                                                indicator_pattern=row ['indicator_pattern'],
                                                                createdByRef = by,
                                                                )
                else:
                      indicator = opencti_api_client.indicator.create(
                                                                name=row['name'],
                                                                description= row['description'],
                                                                pattern_type= row['pattern_type'],
                                                                indicator_pattern= row ['main_observable_type'],
                                                                main_observable_type="File",
                                                                createdByRef = by,
                                                                update =True
                                                                  )
                if row['marking_definitions'] is not None:
                        self.add_mark_def(stix_id=vulnerability['id'],row=row)
                        
                opencti_api_client.report.add_stix_entity(id=report,entity_id = indicator['id'] )
                self.stix_domain.update ({row['name'].strip() : indicator['id']})  
                self.add_external_references_to_stix_entity (url=row['external_references'],entity_id =indicator['id'],ext_ref =self.ext_ref ) 
                

        
    def create_relationships(self,report):

        relation_details= pd.read_excel(self.relation_path,sheet_name='relations')
        relation_details = relation_details.where(relation_details.notnull(), None)

        df=relation_details.copy()
        df['to'] = [str(i).strip() for i in df['to']]
        df['to']=df['to'].map(self.stix_domain)
        df['from']=df['from'].map(self.stix_domain)

        df = df.where(df.notnull(), None)


        for index,row in df.iterrows():

            if ((row['from']) is not None ) & ((row['to']) is not None ):

                    print(row['from'])
                    print(row['to'])
                    relation = opencti_api_client.stix_relation.create(
                    fromType= opencti_api_client.get_stix_domain_entity (id =row['from'])['entity_type'],
                    fromId=row['from'],
                    relationship_type = row['relation'],
                    toType=opencti_api_client.get_stix_domain_entity (row['to'])['entity_type'],
                    toId= row ['to'],
                    )

                    opencti_api_client.report.add_stix_entity(id=report,entity_id = relation['id'] )

    
    def create_report(self):

        report_details = pd.read_excel(self.report_path,sheet_name = 'report')
        report_details = report_details.where(report_details.notnull(), None)
        
        #report creation
        for index,row in report_details.iterrows():
            report_by = opencti_api_client.identity.list(filters=[{'key':'name','values': row['created_by_ref'] }])
           
            if len(report_by) == 0 : 
                warnings.warn("Since " + row['created_by_ref'] + " is not inserted in opencti.Rhymtechnologies is taken as default")
                report_by = organization = opencti_api_client.identity.create(
                                            type='Organization',
                                            name="Rhymtechnologies",
                                            alias=["Rhymtech"],
                                            description="A young and innovative company formed with a vision to create world leading Cyber Security products and services. The promoters are a group of highly successful professionals with diverse backgrounds across IT Delivery, Sales and Consulting, Cyber Security, Artificial Intelligence, Machine Learning and Data Sciences. Our group of consultants led by some of the best minds in the industry have successfully provided strategic and value-add services for multiple large organizations globally.",
                                            )
            else:
                report_by = report_by[0]
                
            #report_by = {'id': None}
                
            
            print(row ['source_confidence_level'])
            
            report = opencti_api_client.report.create(
                                                            name=row['name'],
                                                            description= row['description'],
                                                            report_class="Internal Report",
                                                            published=datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00"),
                                                            createdByRef= report_by['id'],
                                                            object_status = row['object_status'],
                                                            source_confidence_level = row ['source_confidence_level'],
                                                            update = True,
                                                            )
            if row['marking_definitions'] is not None:
                        self.add_mark_def(stix_id=threat_actor['id'],row=row)
            print("one_completed")
            
            self.creating_and_adding_ext_ref(by = report_by['id'] ,report = report['id'])  
            self.create_threat_actor(by = report_by['id'] ,report = report['id'])  
            self.create_attack_pattern(by = report_by['id'] , report =report ['id'])
            self.create_malware (by = report_by['id'] , report = report ['id'])
            self.create_tool (by = report_by['id'] , report = report ['id'])
            self.create_vulnerabilty (by = report_by['id'] , report = report ['id'])
            self.create_indicator (by = report_by['id'] , report = report ['id'])
            self.create_relationships (report = report ['id'])
            
            
            return [self.stix_domain,self.ext_ref]
                 
                
        

In [127]:
en=entity_insertion(report_path=r"D:\Threat_intel_services\srikanth\fakeapt28.xlsx",relation_path=r"D:\Threat_intel_services\srikanth\fakeapt28_relations.xlsx")

In [128]:
en.create_report()

INFO:root:Listing Identities with filters [{"key": "name", "values": "Rhymtech"}].
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:320: UserWarning: Since Rhymtech is not inserted in opencti.Rhymtechnologies is taken as default
INFO:root:Listing Stix-Domain-Entities with filters [{"key": "name", "values": ["Rhymtechnologies"]}].
INFO:root:Listing Reports with filters [{"key": "name", "values": ["fakeapt28"]}, {"key": "published_day", "values": ["2020-02-04"]}].
INFO:root:Creating Report {fakeapt28}.


1


INFO:root:Listing External-Reference with filters [{"key": "url", "values": ["link:https://www.crowdstrike.com/blog/whois-numbered-panda/"]}].
INFO:root:Reading Stix-Entity {1f2cbc39-8893-4bec-9c59-b9f8fba67062}.


one_completed


INFO:root:Adding External-Reference {b83dd90a-af30-4141-a907-57a90916bf1c} to Stix-Entity {1f2cbc39-8893-4bec-9c59-b9f8fba67062}
INFO:root:Listing External-Reference with filters [{"key": "url", "values": ["https://www.thaicert.or.th/downloads/files/A_Threat_Actor_Encyclopedia.pdf"]}].
INFO:root:Reading Stix-Entity {1f2cbc39-8893-4bec-9c59-b9f8fba67062}.
INFO:root:Adding External-Reference {df04121c-2152-4fd3-aba7-b91ac44027af} to Stix-Entity {1f2cbc39-8893-4bec-9c59-b9f8fba67062}
INFO:root:Listing External-Reference with filters [{"key": "url", "values": ["https://attack.mitre.org/groups/G0005/"]}].
INFO:root:Reading Stix-Entity {1f2cbc39-8893-4bec-9c59-b9f8fba67062}.
INFO:root:Adding External-Reference {0b6149a7-74f6-4b59-8549-7774595335fc} to Stix-Entity {1f2cbc39-8893-4bec-9c59-b9f8fba67062}
INFO:root:Listing External-Reference with filters [{"key": "url", "values": ["https://www.youtube.com/watch?v=dy72wfyXlrM"]}].
INFO:root:Reading Stix-Entity {1f2cbc39-8893-4bec-9c59-b9f8fba6706

...

7fc83aa3-8c7a-40da-8cf4-0b9b90c4c0ac
d380cf57-9810-438b-bb25-a652aaca9ab2


INFO:root:Listing stix_relations with {type: indicates, from_id: 7fc83aa3-8c7a-40da-8cf4-0b9b90c4c0ac, to_id: d380cf57-9810-438b-bb25-a652aaca9ab2}
INFO:root:Reading Report {1f2cbc39-8893-4bec-9c59-b9f8fba67062}.
INFO:root:Adding Stix-Entity {b570b5b8-4d12-4a42-8ec5-740729eb486a} to Report {1f2cbc39-8893-4bec-9c59-b9f8fba67062}
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:300: DeprecationWarning: Call to deprecated method get_stix_domain_entity. (Replaced by the StixDomainEntity class in pycti) -- Deprecated since version 2.1.0.
INFO:root:Reading Stix-Domain-Entity {be19e5c7-c324-4215-adc8-0479a3d6c21c}.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:303: DeprecationWarning: Call to deprecated method get_stix_domain_entity. (Replaced by the StixDomainEntity class in pycti) -- Deprecated since version 2.1.0.
INFO:root:Reading Stix-Domain-Entity {f14a312d-c43b-4c07-9951-f3448dfb7ba6}.


be19e5c7-c324-4215-adc8-0479a3d6c21c
f14a312d-c43b-4c07-9951-f3448dfb7ba6


INFO:root:Listing stix_relations with {type: indicates, from_id: be19e5c7-c324-4215-adc8-0479a3d6c21c, to_id: f14a312d-c43b-4c07-9951-f3448dfb7ba6}
INFO:root:Reading Report {1f2cbc39-8893-4bec-9c59-b9f8fba67062}.
INFO:root:Adding Stix-Entity {ef76b9dd-b8e8-4906-bbab-f3d2d7528d71} to Report {1f2cbc39-8893-4bec-9c59-b9f8fba67062}


[{'fakeapt28': '22175c83-6fb7-4f81-b2e6-8e296a6ac451',
  'Buy domain name': 'f98a818c-d149-4bb5-8963-2fe7ce605986',
  'Obtain/re-use payloads': 'a2c6b37c-fa8f-4537-b548-9294727d2ca2',
  'Access Token Manipulation': '830843c5-2a10-4e76-8b65-4111878d0925',
  'Application Access Token': 'ec1b152b-759b-4656-8417-fd9caebf312b',
  'Automated Collection': '77ac5165-2682-46c7-b86c-76c5d2c38a4e',
  'Bootkit': '0ba1b7ae-d989-4b6b-b90e-75be1438e8d0',
  'Command-Line Interface': '885223eb-a293-4d4a-aaa8-2dd8d4487076',
  'Commonly Used Port': '651e2539-4a7d-47fc-8df2-ff67583c3801',
  'Communication Through Removable Media': '6f01d49b-f9aa-4610-8bf2-246f9d3e4875',
  'Component Object Model Hijacking': 'ba4ce101-a595-428e-b56b-a7fd085ab3ed',
  'Connection Proxy': '8c57bb26-ccef-4301-83cb-6a0ae1bcc508',
  'Credential Dumping': '9c364890-7132-4d41-b324-7a4e44aad1f6',
  'Custom Cryptographic Protocol': '923139f5-b823-482c-8fd9-0f0e49672e78',
  'Data Compressed': '1f010009-f6eb-4223-a098-807e6384a1ae',
 

In [125]:
en.create_relationships(report = 'a216498d-1d4c-4555-a6a5-c62f38300e74')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:300: DeprecationWarning: Call to deprecated method get_stix_domain_entity. (Replaced by the StixDomainEntity class in pycti) -- Deprecated since version 2.1.0.
INFO:root:Reading Stix-Domain-Entity {22175c83-6fb7-4f81-b2e6-8e296a6ac451}.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:303: DeprecationWarning: Call to deprecated method get_stix_domain_entity. (Replaced by the StixDomainEntity class in pycti) -- Deprecated since version 2.1.0.
INFO:root:Reading Stix-Domain-Entity {f98a818c-d149-4bb5-8963-2fe7ce605986}.


22175c83-6fb7-4f81-b2e6-8e296a6ac451
f98a818c-d149-4bb5-8963-2fe7ce605986


INFO:root:Listing stix_relations with {type: uses, from_id: 22175c83-6fb7-4f81-b2e6-8e296a6ac451, to_id: f98a818c-d149-4bb5-8963-2fe7ce605986}
INFO:root:Reading Report {a216498d-1d4c-4555-a6a5-c62f38300e74}.
INFO:root:Adding Stix-Entity {592edc65-4cb8-4d59-a41d-6ec9fb7e87e6} to Report {a216498d-1d4c-4555-a6a5-c62f38300e74}
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:300: DeprecationWarning: Call to deprecated method get_stix_domain_entity. (Replaced by the StixDomainEntity class in pycti) -- Deprecated since version 2.1.0.
INFO:root:Reading Stix-Domain-Entity {22175c83-6fb7-4f81-b2e6-8e296a6ac451}.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:303: DeprecationWarning: Call to deprecated method get_stix_domain_entity. (Replaced by the StixDomainEntity class in pycti) -- Deprecated since version 2.1.0.
INFO:root:Reading Stix-Domain-Entity {a2c6b37c-fa8f-4537-b548-9294727d2ca2}.


22175c83-6fb7-4f81-b2e6-8e296a6ac451
a2c6b37c-fa8f-4537-b548-9294727d2ca2


INFO:root:Listing stix_relations with {type: uses, from_id: 22175c83-6fb7-4f81-b2e6-8e296a6ac451, to_id: a2c6b37c-fa8f-4537-b548-9294727d2ca2}
INFO:root:Reading Report {a216498d-1d4c-4555-a6a5-c62f38300e74}.
INFO:root:Adding Stix-Entity {a5369d2f-938d-4327-8bc8-84d1d9c9c06b} to Report {a216498d-1d4c-4555-a6a5-c62f38300e74}
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:300: DeprecationWarning: Call to deprecated method get_stix_domain_entity. (Replaced by the StixDomainEntity class in pycti) -- Deprecated since version 2.1.0.
INFO:root:Reading Stix-Domain-Entity {22175c83-6fb7-4f81-b2e6-8e296a6ac451}.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:303: DeprecationWarning: Call to deprecated method get_stix_domain_entity. (Replaced by the StixDomainEntity class in pycti) -- Deprecated since version 2.1.0.
INFO:root:Reading Stix-Domain-Entity {830843c5-2a10-4e76-8b65-4111878d0925}.


...

INFO:root:Listing stix_relations with {type: indicates, from_id: 8f5bc80d-30ca-4c10-b294-27dad63d3061, to_id: d380cf57-9810-438b-bb25-a652aaca9ab2}
INFO:root:Reading Report {a216498d-1d4c-4555-a6a5-c62f38300e74}.
INFO:root:Adding Stix-Entity {7d8127b7-8924-43dc-9c78-7ead43377217} to Report {a216498d-1d4c-4555-a6a5-c62f38300e74}
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:300: DeprecationWarning: Call to deprecated method get_stix_domain_entity. (Replaced by the StixDomainEntity class in pycti) -- Deprecated since version 2.1.0.
INFO:root:Reading Stix-Domain-Entity {7fc83aa3-8c7a-40da-8cf4-0b9b90c4c0ac}.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:303: DeprecationWarning: Call to deprecated method get_stix_domain_entity. (Replaced by the StixDomainEntity class in pycti) -- Deprecated since version 2.1.0.
INFO:root:Reading Stix-Domain-Entity {d380cf57-9810-438b-bb25-a652aaca9ab2}.


7fc83aa3-8c7a-40da-8cf4-0b9b90c4c0ac
d380cf57-9810-438b-bb25-a652aaca9ab2


INFO:root:Listing stix_relations with {type: indicates, from_id: 7fc83aa3-8c7a-40da-8cf4-0b9b90c4c0ac, to_id: d380cf57-9810-438b-bb25-a652aaca9ab2}
INFO:root:Reading Report {a216498d-1d4c-4555-a6a5-c62f38300e74}.
INFO:root:Adding Stix-Entity {b570b5b8-4d12-4a42-8ec5-740729eb486a} to Report {a216498d-1d4c-4555-a6a5-c62f38300e74}
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:300: DeprecationWarning: Call to deprecated method get_stix_domain_entity. (Replaced by the StixDomainEntity class in pycti) -- Deprecated since version 2.1.0.
INFO:root:Reading Stix-Domain-Entity {be19e5c7-c324-4215-adc8-0479a3d6c21c}.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:303: DeprecationWarning: Call to deprecated method get_stix_domain_entity. (Replaced by the StixDomainEntity class in pycti) -- Deprecated since version 2.1.0.
INFO:root:Reading Stix-Domain-Entity {f14a312d-c43b-4c07-9951-f3448dfb7ba6}.


be19e5c7-c324-4215-adc8-0479a3d6c21c
f14a312d-c43b-4c07-9951-f3448dfb7ba6


INFO:root:Listing stix_relations with {type: indicates, from_id: be19e5c7-c324-4215-adc8-0479a3d6c21c, to_id: f14a312d-c43b-4c07-9951-f3448dfb7ba6}
INFO:root:Reading Report {a216498d-1d4c-4555-a6a5-c62f38300e74}.
INFO:root:Adding Stix-Entity {ef76b9dd-b8e8-4906-bbab-f3d2d7528d71} to Report {a216498d-1d4c-4555-a6a5-c62f38300e74}


In [30]:
opencti_api_client.stix_observable.create(type='File-MD5',observable_value='16a9f340c0d353332ba6f525376c93e1')

INFO:root:Listing StixObservables with filters [{"key": "observable_value", "values": ["16a9f340c0d353332ba6f525376c93e1"]}].


{'id': 'f83d95e4-3540-4417-bbcd-b422bdb45d93',
 'stix_id_key': 'observable--5506469d-f820-4ddb-8edd-4cf04e6457aa',
 'entity_type': 'file',
 'name': '',
 'description': None,
 'observable_value': '16a9f340c0d353332ba6f525376c93e1',
 'created_at': '2020-02-01T12:16:25.666Z',
 'updated_at': '2020-02-01T12:16:25.666Z',
 'createdByRef': None,
 'tags': [],
 'markingDefinitions': [],
 'externalReferences': [],
 'indicators': [],
 'markingDefinitionsIds': [],
 'tagsIds': [],
 'externalReferencesIds': [],
 'indicatorsIds': []}

In [57]:
df=pd.read_excel(r"D:\Threat_intel_services\srikanth\fakeapt12.xlsx",sheet_name = 'Indicator' ) 

In [ ]:
indicator = opencti_api_client.indicator.create(
    name=row['name'],
    description= row['description'],
    pattern_type= row['pattern_type'],
    indicator_pattern= row ['main_observable_type'],
    main_observable_type="File",
)

In [122]:
opencti_api_client.indicator.read( id ='020f02e8-75d1-4527-8000-19fcf34a2b1c')

INFO:root:Reading Indicator {020f02e8-75d1-4527-8000-19fcf34a2b1c}.
ERROR:root:An unknown error has occurred!  Please try again later.


TypeError: 'NoneType' object is not subscriptable

In [93]:
opencti_api_client.indicator.read(id = '2fddbc1b-a3db-4156-8bf5-839f2c54ada9')


INFO:root:Reading Indicator {2fddbc1b-a3db-4156-8bf5-839f2c54ada9}.


{'id': '2fddbc1b-a3db-4156-8bf5-839f2c54ada9',
 'stix_id_key': 'indicator--93bc28bc-a1bb-40be-920a-9bbb34a5509e',
 'stix_label': None,
 'entity_type': 'indicator',
 'parent_types': ['Indicator', 'Stix-Domain-Entity', 'Stix-Domain'],
 'name': '8.8.8.8',
 'alias': [''],
 'description': 'Simple indicator of observable {8.8.8.8}',
 'graph_data': None,
 'indicator_pattern': "[ipv4-addr:value = '8.8.8.8']",
 'pattern_type': 'stix',
 'valid_from': '2020-01-23T00:51:50.426Z',
 'valid_until': '2021-01-22T00:51:50.426Z',
 'score': 50,
 'created': '2020-01-23T00:51:50.426Z',
 'modified': '2020-01-23T00:51:50.426Z',
 'created_at': '2020-01-23T00:51:50.426Z',
 'updated_at': '2020-01-23T00:51:50.426Z',
 'killChainPhases': [],
 'createdByRef': None,
 'markingDefinitions': [{'id': '41f90184-7fcb-412f-8fa4-567c3346f0d6',
   'entity_type': 'marking-definition',
   'stix_id_key': 'marking-definition--613f2e26-407d-48c7-9eca-b8e91df99dc9',
   'definition_type': 'TLP',
   'definition': 'TLP:WHITE',
   'lev

In [ ]:
main_dict={'fakeapt12': 'f55ab06f-da9c-417f-bcd5-5127e4bd459b',
  'Spear phishing': 'fa1fd76f-4535-45e3-9e0b-86b3c5b98a24',
  'Exploitation for Client Execution': 'ed27d3c8-e054-49ff-80d1-19c0d72dc286',
  'Spearphishing attachment': '3a2e89b6-4dbd-4172-8340-b424dff022f9',
  'User Execution': 'aa4dac29-3d7a-449c-9455-28977b7f27a2',
  'Web Service': '3b5277c0-34c5-4371-8516-3aeee9c14d69',
  'Command-Line Interface': '7aa0b906-0670-4d97-a0f3-00fb626856d4',
  'Commonly Used Port': 'fab5312a-b85a-4b52-961c-a18deed6a0ef',
  'Data from Local System': '97d127c9-761b-422f-b2af-f162779e2c72',
  'Data Obfuscation': 'd1aa53d0-9808-4325-b46a-612b83212b4a',
  'File and Directory Discovery': '3b6464bf-890d-494a-835f-f5f8cc4b2d58',
  'File Deletion': '5d0c505b-999d-4039-8e18-a704b234f972',
  'Hidden Files and Directories': '7df7f45d-47f9-4e60-b984-a12cfcd4e034',
  'Masquerading': 'ba2deef8-4c2f-4346-b412-36a148697fec',
  'Process Discovery': '637c1675-6cff-479b-abf2-a6828ee06f49',
  'Registry Run Keys / Startup Folder': 'cdc06ba5-99f3-4023-9534-2aa2e2d033f6',
  'Remote File Copy': 'bffaec7e-4751-41d7-a700-03d3131e6ce6',
  'Standard Application Layer Protocol': '746fa63c-2c4a-48aa-b7a9-50433725ecd7',
  'System Information Discovery': '0f4235bc-d348-4228-bf8c-59bef3d57c6c',
  'System Network Configuration Discovery': '63d0b6b7-3720-42f4-96ab-d6e46c3c133d',
  'System Owner/User Discovery': 'c6728d7c-2822-43ff-be59-80b7d6c5a033',
  'System Service Discovery': '7f54a4d8-f454-4a48-a510-be07ff5d2f93',
  'Standard Cryptographic Protocol': '213b60fb-44e3-46f0-ab02-faae8ce62653',
  'Connection Proxy': '079578de-fe65-45f2-b0d7-882f8f0b1f4b',
  'Process Injection': '06e637be-7ca5-4750-8ac3-de6d82fdd174',
  'Rootkit': 'b7e54f8e-1390-4eec-81fa-7fe83f7b8b18',
  'Ixeshe': '96d9e9c1-c06c-490f-9453-d578c93e14ba',
  'Aumlib': '5cf859fe-b7bc-4a2a-a7f1-6a7e22dd00cf',
  'Threebyte': '6d53e9e7-9c6f-43a7-ad6c-1def2eee030e',
  'Hightide': '8a91cd27-e581-45e8-bf55-26084bcfdaad',
  'Waterspout': 'f15bce7b-e2cd-49e9-8001-2d6fdd94d4e0',
  'HTRAN': 'ad08da3f-5f3c-44a6-81ff-9082ef710e4c',
  'CVE-2009-4324': '7b22dff2-486b-402b-b338-c67950a59126',
  'CVE-2009-0927': '671c5891-435c-416d-8576-8b4e7237a354',
  'CVE-2011-0609': '116e96f5-675e-4271-ad95-ab324db18dce',
  'CVE-2011-0611': '2c0765b3-c1ba-4e19-81a5-0149be5d55b2',
  'CVE-2009-3129': '3f3c06e1-c26d-447f-917d-70d8c998b284',
  'CVE-2012-0158': '95e3a6fc-aba3-432e-9a33-3be79d0b5e62'}

In [26]:
intrusion_set = opencti_api_client.intrusion_set.create(
    name="My new Intrusion Set",
    description="Evil Cluster",
    first_seen=datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00"),
    last_seen=datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00"),
    update=True,
)

INFO:root:Listing Stix-Domain-Entities with filters [{"key": "name", "values": ["My new Intrusion Set"]}].
INFO:root:{"errors":[{"message":"Cannot query field \"observableRefs\" on type \"Incident\".","locations":[{"line":126,"column":17}],"extensions":{"code":"GRAPHQL_VALIDATION_FAILED"}}]}



TypeError: 'NoneType' object is not subscriptable

In [11]:
opencti_api_client.stix_observable.read ( id ='ce4664ee-b323-4b79-a72b-e05ef9278ab6')

INFO:root:Reading StixObservable {ce4664ee-b323-4b79-a72b-e05ef9278ab6}.


{'id': 'ce4664ee-b323-4b79-a72b-e05ef9278ab6',
 'stix_id_key': 'observable--2945ea37-d547-4f0f-b34f-376363422f97',
 'entity_type': 'file-sha256',
 'name': '3603abdb3a307872b8bad338640095b65ca59439a34a372584073ff794a306e3',
 'description': 'Sample',
 'observable_value': '3603abdb3a307872b8bad338640095b65ca59439a34a372584073ff794a306e3',
 'created_at': '2020-02-02T01:59:21.917Z',
 'updated_at': '2020-02-02T01:59:21.917Z',
 'createdByRef': {'id': '2e0562f3-a790-4b18-9696-8a602d29d380',
  'entity_type': 'organization',
  'stix_id_key': 'identity--65a61e6d-d108-44b6-86cb-a60c585d1e5a',
  'stix_label': None,
  'name': 'CIRCL',
  'alias': [''],
  'description': '',
  'created': '2020-01-22T23:40:04.261Z',
  'modified': '2020-01-23T00:00:14.579Z',
  'organization_class': 'vendor',
  'remote_relation_id': '8bbef61e-a0ed-56db-95b2-601a126f7738'},
 'tags': [],
 'markingDefinitions': [{'id': '41f90184-7fcb-412f-8fa4-567c3346f0d6',
   'entity_type': 'marking-definition',
   'stix_id_key': 'marking-

In [ ]:
#add self
def create_relationships(relation_path,report ='2dc633ee-3a27-48c1-900f-58428765ea55'):
    
    relation_details= pd.read_excel(relation_path,sheet_name='threat_actor_relations')
    relation_details = relation_details.where(relation_details.notnull(), None)
    
    df=relation_details.copy()
    df['to'] = [str(i).strip() for i in df['to']]
    df['to']=df['to'].map(main_dict)
    df['from']=df['from'].map(main_dict)
    
    df = df.where(df.notnull(), None)
    
    
    for index,row in df.iterrows():
        
        if ((row['from']) is not None ) & ((row['to']) is not None ):
        
                print(row['from'])
                print(row['to'])
                relation = opencti_api_client.stix_relation.create(
                fromType= opencti_api_client.get_stix_domain_entity (id =row['from'])['entity_type'],
                fromId=row['from'],
                relationship_type = row['relation'],
                toType=opencti_api_client.get_stix_domain_entity (row['to'])['entity_type'],
                toId= row ['to'],
                )
        
                opencti_api_client.report.add_stix_entity(id=report,entity_id = relation['id'] )

In [ ]:
#add self
def create_relationships(relation_path,report ='2dc633ee-3a27-48c1-900f-58428765ea55'):
    
    relation_details= pd.read_excel(relation_path,sheet_name='threat_actor_relations')
    relation_details = relation_details.where(relation_details.notnull(), None)
    
    df=relation_details.copy()
    df['to'] = [str(i).strip() for i in df['to']]
    df['to']=df['to'].map(main_dict)
    df['from']=df['from'].map(main_dict)
    
    df = df.where(df.notnull(), None)
    
    
    for index,row in df.iterrows():
        
        if ((row['from']) is not None ) & ((row['to']) is not None ):
        
                print(row['from'])
                print(row['to'])
                relation = opencti_api_client.stix_relation.create(
                fromType= opencti_api_client.get_stix_domain_entity (id =row['from'])['entity_type'],
                fromId=row['from'],
                relationship_type = row['relation'],
                toType=opencti_api_client.get_stix_domain_entity (row['to'])['entity_type'],
                toId= row ['to'],
                )
        
                opencti_api_client.report.add_stix_entity(id=report,entity_id = relation['id'] )

In [ ]:
def create_tool(self,by,report):           #malware insertion    
            
            tool_details = pd.read_excel(self.report_path,sheet_name = 'tool')
            tool_details = tool_details.where(tool_details.notnull(), None)
            
            for index,row in tool_details.iterrows():
            
                if row['alias'] is not None:
                    aliases =row['alias'].split(',')
                    print(aliases)
                else:
                    aliases = None
                
                exist = opencti_api_client.tool.list ( filters=[{'key' : 'name' , 'values' : row['name']}])
            
                if (len(exist) > 0):
                    tool = opencti_api_client.tool.create(
                                                name = row['name'],
                                                description = row['description'],
                                                alias = aliases ,
                                                modified = datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00") ,
                                                createdByRef = by,
                                                )
                else:
                     tool = opencti_api_client.tool.create(
                                                name = row['name'],
                                                description = row['description'],
                                                alias = aliases ,
                                                modified = datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00") ,
                                                update = True,
                                                 )
                if row['marking_definitions'] is not None:
                        self.add_mark_def(stix_id=tool['id'],row=row)
                        
                opencti_api_client.report.add_stix_entity(id=report,entity_id = tool['id'] )

In [ ]:
def create_report(self,report_by):

    report_details=pd.read_excel(self.report_path,sheet_name = 'report')
    report_by = opencti_api_client.identity.read(filters=[{'key':'name','values': report_by }])
    
    if report_by is None: 
        warnings.warn("Since " + report_by + " is not inserted in opencti.Rhymtechnologies is taken as default")
        report_by = organization = opencti_api_client.identity.create(
                                    type='Organization',
                                    name="Rhymtechnologies",
                                    alias=["Rhymtech"],
                                    description="A young and innovative company formed with a vision to create world leading Cyber Security products and services. The promoters are a group of highly successful professionals with diverse backgrounds across IT Delivery, Sales and Consulting, Cyber Security, Artificial Intelligence, Machine Learning and Data Sciences. Our group of consultants led by some of the best minds in the industry have successfully provided strategic and value-add services for multiple large organizations globally.",
                                    )
                        
    
    
    for index,row in report_details.iterrows():
        
        date = parse(row['published']).strftime("%Y-%m-%dT%H:%M:%SZ")
        if source_confidence_level is None:
                    report = opencti_api_client.report.create(
                                                    name=row['name'],
                                                    description= row['description'],
                                                    report_class="Internal Report",
                                                    published=date,
                                                    createdByRef= report_by['id'],
                                                    object_status = 0,
                                                    update = True,
                                                    )
        else:
             report = opencti_api_client.report.create(
                                                    name=row['name'],
                                                    description= row['description'],
                                                    report_class="Internal Report",
                                                    published=date,
                                                    createdByRef= report_by['id'],
                                                    object_status = row['object_status'],
                                                    source_confidence_level = row ['source_confidence_level'],
                                                    update = True,
                                                    )


In [ ]:
create_report()